In [1]:
!pip install transformers datasets wandb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from huggingface_hub import login
#removed login for security
login("")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer
import pandas as pd
from datasets import Dataset
import wandb
import os
import gc

# Load the dataset
data_path = 'demo.xlsx'  # Your file path
df = pd.read_excel(data_path, nrows=500)  # Smaller subset to fit in memory
df = df[['Question', 'Answer']]
dataset = Dataset.from_pandas(df)

# Load the smaller GPT-2 model and tokenizer
model_name = "gpt2"  # GPT-2 small model
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Set padding token
tokenizer.pad_token = tokenizer.eos_token

# Define tokenization function
def tokenize_function(examples):
    inputs = tokenizer(
        [f"{q} {a}" for q, a in zip(examples["Question"], examples["Answer"])],
        padding="max_length",
        truncation=True,
        max_length=128  # Adjust max_length as needed
    )
    inputs["labels"] = inputs["input_ids"]
    return inputs

# Split the dataset
train_test_split = dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

# Tokenize
train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["Question", "Answer"])
eval_dataset = eval_dataset.map(tokenize_function, batched=True, remove_columns=["Question", "Answer"])

# Set training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    load_best_model_at_end=True,
    save_strategy="epoch",
    report_to="wandb",
    gradient_checkpointing=True,
    fp16=True,
)

# Initialize W&B
wandb.login()
wandb.init(project="your_project_name")

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Start training
try:
    trainer.train()
except Exception as e:
    print(f"Error during training: {e}")

# Clear unused variables
del df, dataset, train_dataset, eval_dataset
gc.collect()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dpatel21 (dpatel21-michigan-technological-university). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants

Epoch,Training Loss,Validation Loss
1,No log,0.258724
2,0.754600,0.191977
3,0.190500,0.184052


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
There were missing keys in the checkpoint model loaded

33

In [ ]:
import torch

# Assuming the model is already loaded
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def generate_answer(question):
    # Tokenize the question
    inputs = tokenizer(question, return_tensors="pt").to(device)  # Move to the same device
    # Generate an answer using the model
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=200,  # Set the maximum length for the generated text
            min_length=100,  # Set the minimum length for the generated text
            do_sample=True,   # Use sampling to generate more diverse responses
            top_k=50,        # Limit the sampling to the top 50 words
            top_p=0.95,      # Use nucleus sampling
            num_return_sequences=1,  # Generate one sequence
        )
    # Decode the generated tokens
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Generate an answer
question = "Are CPIs adjusted for different locations?"
answer = generate_answer(question)
print("Generated answer:", answer)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated answer: Are CPIs adjusted for different locations? Consumer price indexes (CPIs) are index numbers that measure changes in the prices of goods and services purchased or otherwise acquired by households, which households use directly, or indirectly, to satisfy their own needs and wants. In practice, most CPIs are calculated as weighted averages of the percentage price changes for a specified set, or “basket”, of consumer products, the weights reflecting their relative importance in household consumption in some period. CPIs are widely used to index pensions and social security benefits. CPIs are also used to index other payments, such as interest payments or rents, or the prices of financial assets and liabilities. CPIs are also used to index other payments, such as interest payments or rents, or the prices of financial assets and liabilities. CPIs are also used to index other payments, such as interest payments or rents, or the prices of financial assets and liabilities. CPIs

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
import pandas as pd
from datasets import Dataset
import wandb
import os
import gc

# Load the dataset
data_path = 'demo.xlsx'  # Your file path
df = pd.read_excel(data_path, nrows=100)  # Smaller subset to fit in memory
df = df[['Question', 'Answer']]
dataset = Dataset.from_pandas(df)

# Load the smaller openelm model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-360M-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Set padding token
tokenizer.pad_token = tokenizer.eos_token

# Define tokenization function
def tokenize_function(examples):
    inputs = tokenizer(
        [f"{q} {a}" for q, a in zip(examples["Question"], examples["Answer"])],
        padding="max_length",
        truncation=True,
        max_length=128  # Adjust max_length as needed
    )
    inputs["labels"] = inputs["input_ids"]
    return inputs

# Split the dataset
train_test_split = dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

# Tokenize
train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["Question", "Answer"])
eval_dataset = eval_dataset.map(tokenize_function, batched=True, remove_columns=["Question", "Answer"])

# Set training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    load_best_model_at_end=True,
    save_strategy="epoch",
    report_to="wandb",
    gradient_checkpointing=True,
    # fp16=True,
    bf16=True,
)

# Initialize W&B
wandb.login()
wandb.init(project="your_project_name")

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Start training
try:
    trainer.train()
except Exception as e:
    print(f"Error during training: {e}")

# Clear unused variables
del df, dataset, train_dataset, eval_dataset
gc.collect()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.76k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

...

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dpatel21 (dpatel21-michigan-technological-university). Use `wandb login --relogin` to force relogin


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,No log,0.559698
2,No log,0.318741
3,No log,0.269745


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommend

62

In [4]:
import torch

# Assuming the model is already loaded
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def generate_answer(question):
    # Tokenize the question
    inputs = tokenizer(question, return_tensors="pt").to(device)  # Move to the same device
    # Generate an answer using the model
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=200,  # Set the maximum length for the generated text
            min_length=100,  # Set the minimum length for the generated text
            do_sample=True,   # Use sampling to generate more diverse responses
            top_k=50,        # Limit the sampling to the top 50 words
            top_p=0.95,      # Use nucleus sampling
            num_return_sequences=1,  # Generate one sequence
        )
    # Decode the generated tokens
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Generate an answer
question = "Are CPIs adjusted for different locations?"
answer = generate_answer(question)
print("Generated answer:", answer)

Generated answer: Are CPIs adjusted for different locations? Consumer price indexes (CPIs) are index numbers that measure changes in the prices of goods and services purchased or otherwise acquired by households, which households use directly, or indirectly, to satisfy their own needs and wants. In practice, most CPIs are calculated as weighted averages of the percentage price changes for a specified set, or “basket”, of consumer products, the weights reflecting their relative importance in household consumption in some period. CPIs are widely used to index pensions and social security benefits. CPIs are also used to index other payments, such as interest payments or rents, or the prices of bonds. CPIs are also used to index other payments, such as the value of charitable donations or the value of unpaid work conducted by household members, although the relevance of the index to this purpose is generally less clear. CPIs are also used to index the prices ofcessions or the value of publ

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
import pandas as pd
from datasets import Dataset
import wandb
import os
import gc

# Load the dataset
data_path = 'demo.xlsx'  # Your file path
df = pd.read_excel(data_path, nrows=100)  # Smaller subset to fit in memory
df = df[['Question', 'Answer']]
dataset = Dataset.from_pandas(df)

# Load the smaller openelm model and tokenizer
model_name = "facebook/opt-350m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Set padding token
tokenizer.pad_token = tokenizer.eos_token

# Define tokenization function
def tokenize_function(examples):
    inputs = tokenizer(
        [f"{q} {a}" for q, a in zip(examples["Question"], examples["Answer"])],
        padding="max_length",
        truncation=True,
        max_length=128  # Adjust max_length as needed
    )
    inputs["labels"] = inputs["input_ids"]
    return inputs

# Split the dataset
train_test_split = dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

# Tokenize
train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["Question", "Answer"])
eval_dataset = eval_dataset.map(tokenize_function, batched=True, remove_columns=["Question", "Answer"])

# Set training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    load_best_model_at_end=True,
    save_strategy="epoch",
    report_to="wandb",
    gradient_checkpointing=True,
    # fp16=True,
    bf16=True,
)

# Initialize W&B
wandb.login()
wandb.init(project="your_project_name")

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Start training
try:
    trainer.train()
except Exception as e:
    print(f"Error during training: {e}")

# Clear unused variables
del df, dataset, train_dataset, eval_dataset
gc.collect()


tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

...

eval/loss,█▂▁
eval/runtime,▅▁█
eval/samples_per_second,▄█▁
eval/steps_per_second,▄█▁
train/epoch,▁▅██
train/global_step,▁▅██
eval/loss,0.26975
eval/runtime,0.8413
eval/samples_per_second,11.886
eval/steps_per_second,11.886
total_flos,65242755072000.0


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,No log,0.207669
2,No log,0.173939
3,No log,0.181628


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
There were missing keys in the checkpoint model loaded

7234

In [6]:
import torch

# Assuming the model is already loaded
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def generate_answer(question):
    # Tokenize the question
    inputs = tokenizer(question, return_tensors="pt").to(device)  # Move to the same device
    # Generate an answer using the model
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=200,  # Set the maximum length for the generated text
            min_length=100,  # Set the minimum length for the generated text
            do_sample=True,   # Use sampling to generate more diverse responses
            top_k=50,        # Limit the sampling to the top 50 words
            top_p=0.95,      # Use nucleus sampling
            num_return_sequences=1,  # Generate one sequence
        )
    # Decode the generated tokens
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Generate an answer
question = "Are CPIs adjusted for different locations?"
answer = generate_answer(question)
print("Generated answer:", answer)

Generated answer: Are CPIs adjusted for different locations? Consumer price indexes (CPIs) are index numbers that measure changes in the prices of goods and services purchased or otherwise acquired by households, which households use directly, or indirectly, to satisfy their own needs and wants. In practice, most CPIs are calculated as weighted averages of the percentage price changes for a specified set, or “basket”, of consumer products, the weights reflecting their relative importance in household consumption in some period. CPIs are widely used to index pensions and social security benefits. CPIs are also used to index other payments, such as interest payments or rents, or the amounts paid or otherwise acquired by households to fund their own lives. CPIs are widely used to index other payments, such as interest payments or rents, or the amounts paid or otherwise acquired by households to fund their own lives. CPIs are widely used to index other payments, such as interest payments o

In [4]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer
from datasets import load_dataset
import wandb
import gc

# Load the dataset directly from Hugging Face
dataset = load_dataset("FinGPT/fingpt-fiqa_qa")

# Select and preprocess the necessary columns
dataset = dataset.rename_column("input", "Question").rename_column("output", "Answer")
dataset = dataset.remove_columns([col for col in dataset.column_names["train"] if col not in ["Question", "Answer"]])

# Load the smaller GPT-2 model and tokenizer
model_name = "gpt2"  # GPT-2 small model
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Set padding token
tokenizer.pad_token = tokenizer.eos_token

# Define tokenization function
def tokenize_function(examples):
    inputs = tokenizer(
        [f"{q} {a}" for q, a in zip(examples["Question"], examples["Answer"])],
        padding="max_length",
        truncation=True,
        max_length=128
    )
    inputs["labels"] = inputs["input_ids"]
    return inputs

# Split the dataset
train_test_split = dataset["train"].train_test_split(test_size=0.1)
train_dataset = train_test_split['train'].map(tokenize_function, batched=True)
eval_dataset = train_test_split['test'].map(tokenize_function, batched=True)

# Set training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    load_best_model_at_end=True,
    save_strategy="epoch",
    report_to="wandb",
    gradient_checkpointing=True,
    fp16=True,
)

# Initialize W&B
wandb.login()
wandb.init(project="your_project_name")

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Start training
try:
    trainer.train()
except Exception as e:
    print(f"Error during training: {e}")

# Clear unused variables
del train_dataset, eval_dataset
gc.collect()


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

...

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dpatel21 (dpatel21-michigan-technological-university). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants

Epoch,Training Loss,Validation Loss
1,3.007900,2.929355
2,2.859700,2.889746
3,2.772800,2.879405


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
There were missing keys in the checkpoint model loaded

93

In [5]:
model_output_dir = "./trained_model"
model.save_pretrained(model_output_dir)
tokenizer.save_pretrained(model_output_dir)

('./trained_model/tokenizer_config.json',
 './trained_model/special_tokens_map.json',
 './trained_model/vocab.json',
 './trained_model/merges.txt',
 './trained_model/added_tokens.json')

In [6]:
import torch

# Assuming the model is already loaded
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def generate_answer(question):
    # Tokenize the question
    inputs = tokenizer(question, return_tensors="pt").to(device)  # Move to the same device
    # Generate an answer using the model
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=200,  # Set the maximum length for the generated text
            min_length=100,  # Set the minimum length for the generated text
            do_sample=True,   # Use sampling to generate more diverse responses
            top_k=50,        # Limit the sampling to the top 50 words
            top_p=0.95,      # Use nucleus sampling
            num_return_sequences=1,  # Generate one sequence
        )
    # Decode the generated tokens
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Generate an answer
question = "Do I need a business credit card?"
answer = generate_answer(question)
print("Generated answer:", answer)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated answer: Do I need a business credit card? [duplicate] Not really. I can't think of a reason you couldn't charge a business credit card, like I'm sure someone there has, since they're a very large organization. It would probably be the case that someone else charged a bunch of business cards in addition to your business, so you'd still have money left. I don't know if it applies to your bank (unless you have a physical address where you would need to physically establish a business), but it's definitely not a reason to get a card. In any case, it's a legitimate business expense for the card issuer to take up such a position.  If it were you, it would make sense to have the card issued by a bank instead of your business to allow you to make the expense from your business. I don't think your answer makes sense to me. I would be surprised if most businesses would have a credit card, or at least one that allows


In [7]:
import shutil
shutil.make_archive("trained_model", "zip", model_output_dir)


'/content/trained_model.zip'